In [1]:
import os
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime
from werkzeug.utils import secure_filename
from app.deepmodels import return_seq2point, aggregate_seq, return_seq2seq, return_dae

c:\Users\stefa\.conda\envs\casestudy2\lib\site-packages\flask_sqlalchemy\__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
2023-06-14 16:42:32,926:INFO:flask_appbuilder.base:Registering class MyIndexView on menu 
2023-06-14 16:42:32,928:INFO:flask_appbuilder.baseviews:Registering route / ('GET',)
2023-06-14 16:42:32,932:INFO:flask_appbuilder.base:Registering class UtilView on menu 
2023-06-14 16:42:32,932:INFO:flask_appbuilder.baseviews:Registering route /back ('GET',)
2023-06-14 16:42:32,935:INFO:flask_appbuilder.base:Registering class LocaleView on menu 
2023-06-14 16:42:32,936:INFO:flask_appbuilder.baseviews:Registering route /lang/<string:locale> ('GET',)
2023-06-14 16:42:32,939:INFO:flask_appbuilder.base:Registering class SecurityApi on menu 
2023-06-14 16:42:32,940:INFO:flask_appbuilder.api:Register

In [3]:
# BASEDIR = os.path.abspath(os.path.dirname(__file__))
ALLOWED_EXTENSIONS = {'csv'}
SAMPLE_PERIOD = 10
WINDOW_SIZE = 99
SUPPORTED_APPLIANCES = {
    "washingmachine":{
        "name":"Washing Machine",
        "seq2point_weights":"seq2point-temp-weights-washing_machine-epoch0.h5",
        "seq2seq_weights":"seq2seq-temp-weights-washing_machine-epoch0.h5",
        "dae_weights":"dae-temp-weights-washing_machine-epoch0.h5"},
    "fridge":{
        "name":"Fridge",
        "seq2point_weights":"seq2point-temp-weights-fridge-epoch0.h5",
        "seq2seq_weights":"seq2seq-temp-weights-fridge-epoch0.h5",
        "dae_weights":"dae-temp-weights-fridge-epoch0.h5"},
    "microwave":{
        "name":"Microwave",
        "seq2point_weights":"seq2point-temp-weights-microwave-epoch0.h5",
        "seq2seq_weights":"seq2seq-temp-weights-microwave-epoch0.h5",
        "dae_weights":"dae-temp-weights-microwave-epoch0.h5"},
    "kettle":{
        "name":"Kettle",
        "seq2point_weights":"seq2point-temp-weights-kettle-epoch0.h5",
        "seq2seq_weights":"seq2seq-temp-weights-kettle-epoch0.h5",
        "dae_weights":"dae-temp-weights-kettle-epoch0.h5"}
        }

In [ ]:
def predict_seq2point(read_csv, write_prediction,db_engine,user_id):

    data = pd.read_csv(read_csv,header=None).rename(columns={0:'timestamp',1:'aggregate'})
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data = data.set_index('timestamp')

    aggregate = data['aggregate']
    mean_agg, std_agg, aggregate = normalise(aggregate)

    aggregate = np.pad(aggregate, (WINDOW_SIZE//2, WINDOW_SIZE//2 +1))
    aggregate = np.array([aggregate[i:i+WINDOW_SIZE] for i in range(len(aggregate)-WINDOW_SIZE)])
    aggregate = np.expand_dims(aggregate, axis=-1)


    model = return_seq2point()
    for appliance in SUPPORTED_APPLIANCES.keys():
        WEIGHTS_PATH = os.path.join(BASEDIR,"weights",SUPPORTED_APPLIANCES[appliance]['seq2point_weights'])
        model.load_weights(WEIGHTS_PATH)
        predict = model.predict(aggregate)

        data[appliance] = np.squeeze(predict)
        data[appliance]=data[appliance].apply(lambda x: 0 if x < 0 else x)
    
    data['user_id'] = user_id


    data.to_csv(write_prediction)
    # update_dataDB(data.reset_index(),db_engine,user_id)

    return True